In [60]:
import pandas as pd
import numpy as np
import time
import datetime
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.metrics import ndcg_score

# Modeling

In [74]:
train = pd.read_csv("cleaned_train.csv")
test = pd.read_csv("cleaned_test.csv")

In [75]:
multi_categorical_features= ["srch_id", "site_id", "visitor_location_country_id", "prop_country_id", "prop_id", "srch_destination_id", 
                             "comp1_rate", "comp1_inv", "comp2_rate", "comp2_inv", "comp3_rate", "comp3_inv", "comp4_rate", "comp4_inv",
                            "comp5_rate", "comp5_inv", "comp6_rate", "comp6_inv", "comp7_rate", "comp7_inv", "comp8_rate", "comp8_inv",
                            "weekday", "month"]


In [77]:
target = 'position' 
train = train.drop(['target_score'], axis=1)

In [78]:
# # stratified split needed for both train-test split and CV
# splitter = GroupShuffleSplit(test_size=0.1, n_splits=3, random_state = 7)
# split = splitter.split(train, groups=train['srch_id'])
# train_inds, test_inds = next(split)

# train_df = train.iloc[train_inds]
# test_df = train.iloc[test_inds]

# X_train = train_df.drop([target], axis=1)
# X_test = test_df.drop([target], axis=1)
# y_train = train_df[target]
# y_test = test_df[target]

In [79]:
model1 = lgb.LGBMRegressor()

In [50]:
# %%time
# model1.fit(X_train, y_train, categorical_feature=multi_categorical_features)


/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


CPU times: user 2min 53s, sys: 12.1 s, total: 3min 5s
Wall time: 1min 9s


LGBMRegressor()

In [51]:
# %%time
# y_pred1 = model1.predict(X_test)
# y_pred1

CPU times: user 19 s, sys: 763 ms, total: 19.8 s
Wall time: 10.8 s


array([0.10653852, 0.05060416, 0.08014264, ..., 0.1740704 , 0.21434546,
       0.31500774])

In [63]:
# ndcg_score([y_test], [y_pred1], k=5)

0.3007851801491499

In [80]:
%%time

# fit the same model on whole train data
model1.fit(train.drop([target], axis=1), train[target], categorical_feature=multi_categorical_features)

/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


CPU times: user 3min 35s, sys: 24.8 s, total: 4min
Wall time: 1min 40s


LGBMRegressor()

In [81]:
%%time
y_pred1 = model1.predict(test)
y_pred1

CPU times: user 3min 10s, sys: 13.4 s, total: 3min 24s
Wall time: 1min 15s


array([17.43246362, 13.38310137, 18.00866427, ...,  9.02347513,
       12.7154563 , 14.5969279 ])

# Submission

- by position

In [82]:
pos = pd.DataFrame(data = y_pred1, columns=['position'])
pos

,posi
0,17.432464
1,13.383101
2,18.008664
3,17.903479
4,12.944099
...,...
4959178,12.982050
4959179,6.602467
4959180,9.023475
4959181,12.715456


In [83]:
pos["srch_id"] = test['srch_id']
pos["prop_id"] = test['prop_id']

In [84]:
result = pos.sort_values(['srch_id','position'])[["srch_id","prop_id"]]

In [85]:
result.to_csv("sub3.csv", index=False)

- by target score

In [68]:
r = pd.DataFrame(data = y_pred1, columns=['target_score'])
r

,target_score
0,0.316331
1,0.459868
2,0.166017
3,0.184937
4,0.341210
...,...
4959178,0.293694
4959179,1.003173
4959180,0.422971
4959181,0.251620


In [69]:
r["srch_id"] = test['srch_id']
r["prop_id"] = test['prop_id']

In [71]:
result = r.sort_values(['srch_id','target_score'])[["srch_id","prop_id"]]

In [72]:
result.head(10)

,srch_id,prop_id
11,1,61632
26,1,128871
6,1,34263
2,1,14142
19,1,89466
28,1,139162
3,1,22393
7,1,37567
27,1,134992
24,1,123675


In [73]:
result.to_csv("sub2.csv", index=False)